In [ ]:
!pip install openai
!pip install PyPDF2
!pip install transformers==4.25.1
!pip install galai
!pip install langchain
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.2 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS


In [19]:
pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00


In [ ]:
import sqlite3
import re
import time
import os
import pandas as pd

In [ ]:
database_location = '/content/gdrive/MyDrive/Database_results'
database_path = os.path.join(database_location, 'summarization.db')
print(database_path)

/content/gdrive/MyDrive/Database_results/Trial_Results.db


In [ ]:
import os
with open("env.txt", "r") as file:
    for row in file:
        key, value = row.strip().split('=', 1)
        os.environ[key] = value

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [18]:
data_prepared_summarization = '/content/gdrive/MyDrive/claims_199.xlsx'
summarization_data_excel = pd.read_excel(data_prepared_summarization)


In [17]:
#print(summarization_data_excel)

In [ ]:
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [ ]:
#works but throws error for most pdfs due to huge size
# import os
# import sqlite3
# import pandas as pd
# import glob
# from PyPDF2 import PdfReader

# def summarize_pdfs_from_folder(folder_path):
#     summaries = []
#
#     for pdf_file_path in glob.glob(folder_path + "/*.pdf"):
#         print("Processing file:", pdf_file_path)  # Debugging: Print the file being processed
#         try:
#
#             loader = PyPDFLoader(pdf_file_path)
#             docs = loader.load_and_split()
#             chain = load_summarize_chain(llm, chain_type="stuff")
#             summary = chain.run(docs)
#             print("Summary for:", pdf_file_path)
#             print(summary)
#             summaries.append(summary)
#         except Exception as e:
#             print(f"Error processing file {pdf_file_path}: {e}")
#     return summaries

# for index, row in summarization_data_excel.iterrows():
#     claim_name = row['Claim']
#     compound_name = row['Compound']
#     claim_number = row['Claim_Number']
#     claim_folder = f'Authorized_Claim_{claim_number}'
#     folder_path = os.path.join('/content/gdrive/MyDrive/Data_Collection', claim_folder)

#     if not os.path.exists(folder_path):
#         print(f"Folder {folder_path} does not exist")
#         continue

#     print(f"Processing folder: {folder_path}")
#     combined_summary = summarize_pdfs_from_folder(folder_path)

#     #Overall Summary
#     embeddings = OpenAIEmbeddings()
#     text_splitter = CharacterTextSplitter(separator = "\n", chunk_size=1000, chunk_overlap = 200, length_function=len,)
#     texts = text_splitter.split_text(str(combined_summary))
#     docsearch = FAISS.from_texts(texts, embeddings)
#     chain = load_qa_chain(OpenAI(), chain_type="stuff")
#     query_overall = "Summarize the given text"

#     docs_overall = docsearch.similarity_search(query_overall)
#     overall_summary = chain.run(input_documents=docs_overall, question=query_overall)
#     print("\nOverall_Summary",overall_summary)

#     #use langchain_new lfor the final result
#     cursor.execute('''
#     INSERT INTO langchain_results(claim_number, claim, compound_name, combined_summary,overall_summary)
#     VALUES (?, ?, ?, ?, ?)
#     ''', (str(claim_number), str(claim_name), str(compound_name), str(combined_summary), str(overall_summary)))
#     conn.commit()

# # Close the database connection
# conn.close()


In [ ]:
#querying based summarization commonly used, better summaries generated
import os
for index, row in summarization_data_excel.iterrows():
        claim_name = row['Claim']
        compound_name = row['Compound']
        claim_number = row['Claim_Number']
        #print(claim_number)

        claim_folder = f'Authorized_Claim_{claim_number}'
        folder_path = f'/content/gdrive/MyDrive/Thesis_Data/{claim_folder}/'


        pdf_files = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

        embeddings = OpenAIEmbeddings()

#Validation check over here - to confirm whether we are indeed readong in pdf files, as this code is designed to process only pdf and may throw an error else.

        if not pdf_files:
            continue
        else:
            claim_summaries = []

            #
            for pdf_file in pdf_files:
                pdf_file_path = os.path.join(folder_path, pdf_file)
                print(f"Processing PDF file: {pdf_file_path}")

                reader = PdfReader(pdf_file_path)

                extracted_raw_data = ''
                for page in reader.pages:
                    text = page.extract_text()
                    if text:
                        extracted_raw_data += text
                text_splitter = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=200, length_function=len)
                texts = text_splitter.split_text(extracted_raw_data)
                if not texts:
                    continue

                embeddings = OpenAIEmbeddings()

                try:
                    summarization_query = FAISS.from_texts(texts, embeddings)
                    summarization_question = "Summarize the given text while retaing all the important information"
                    chain = load_qa_chain(OpenAI(), chain_type="stuff")
                    docs = summarization_query.similarity_search(summarization_question)
                    individual_summary = chain.run(input_documents=docs, question=summarization_question)
                    print("\nIndividual Summarization", individual_summary)

                except Exception as e:

                    if 'Rate limit' in str(e):
                        time.sleep(20)
                        summarization_query = FAISS.from_texts(texts, embeddings)
                        summarization_question = "Summarize the given text while retaing all the important information"
                        docs = summarization_query.similarity_search(summarization_question)
                        individual_summary = chain.run(input_documents=docs, question=summarization_question)

                claim_summaries.append(str(individual_summary))

                def clean_and_combine_paragraphs(claim_summaries):
                  cleaned_summaries = [summary.replace('\n', '').replace('[', '').replace(']', '') for summary in claim_summaries]
                  combined_paragraph = ' '.join(cleaned_summaries)
                  combined_paragraph = re.sub(r'\s+', ' ', combined_paragraph).replace('...', ',')
                  return combined_paragraph

                time.sleep(40)

            combined_summary = clean_and_combine_paragraphs(claim_summaries)

            text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
            texts = text_splitter.split_text(str(combined_summary))
            docsearch = FAISS.from_texts(texts, embeddings)
            chain = load_qa_chain(OpenAI(), chain_type="stuff")

            query_overall_summary = "Summarize the given text"
            docs_overall = docsearch.similarity_search(query_overall_summary)
            Overall_Summary = chain.run(input_documents=docs_overall, question=query_overall_summary)
            print("\nOverall_Summary",Overall_Summary)

            cursor.execute('''
            INSERT INTO langchain_results_new(claim_number, claim, compound_name, combined_summary,overall_summary)
            VALUES (?, ?, ?, ?, ?)
            ''', (str(claim_number), str(claim_name), str(compound_name), str(combined_summary), str(Overall_Summary)))


            conn.commit()
